In [1]:
import sys
import time
import os.path
from glob import glob
from datetime import datetime, timedelta

# data tools
import h5py
import numpy as np

# custom tools
sys.path.insert(0, '/glade/u/home/ksha/WORKSPACE/utils/')
sys.path.insert(0, '/glade/u/home/ksha/WORKSPACE/Analog_BC/')
sys.path.insert(0, '/glade/u/home/ksha/WORKSPACE/Analog_BC/utils/')

import data_utils as du
import graph_utils as gu

from namelist import * 

In [2]:
# graph tools
import cmaps
import cartopy.crs as ccrs
import cartopy.mpl.geoaxes
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as colors
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection

from matplotlib import ticker
import matplotlib.ticker as mticker
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

%matplotlib inline

In [3]:
need_publish = False

# True: publication quality figures
# False: low resolution figures in the notebook

if need_publish:
    dpi_ = fig_keys['dpi']
else:
    dpi_ = 75

# Data

In [4]:
# importing domain information
with h5py.File(save_dir+'BC_domain_info.hdf', 'r') as h5io:
    base_lon = h5io['base_lon'][...]
    base_lat = h5io['base_lat'][...]
    bc_lon = h5io['bc_lon'][...]
    bc_lat = h5io['bc_lat'][...]
    etopo_bc = h5io['etopo_bc'][...]
    land_mask = h5io['land_mask_base'][...]
    land_mask_bc = h5io['land_mask_bc'][...]

In [5]:
# lead = 2
# en = 0
# day_plot = 31

# with h5py.File(save_dir+'NA_SL_info.hdf', 'r') as h5io:
#     w = h5io['W_SL'][bc_inds[0]:bc_inds[1], bc_inds[2]:bc_inds[3]]
# w[w>0.6] = 0.6


# with h5py.File(ERA_dir+'ERA5_GEFS-fcst_{}.hdf'.format(2019), 'r') as h5io:
#     ERA_true = h5io['era_fcst'][..., lead, bc_inds[0]:bc_inds[1], bc_inds[2]:bc_inds[3]]
# ERA_true[..., land_mask_bc] = np.nan

# with h5py.File(REFCST_dir+'SL_CNN_2019_lead{}.hdf'.format(lead), 'r') as h5io:
#     cnn = h5io['cnn_pred'][:, en, ...]
    
# with h5py.File(REFCST_dir + "SL_final_2019_lead{}.hdf".format(lead), 'r') as h5io:
#     sl = h5io['AnEn'][:, en, ...]
#     sl_sg = h5io['AnEn_SG'][:, en, ...]

# sl[..., land_mask_bc] = np.nan
# cnn[..., land_mask_bc] = np.nan

# sl_elev = (1-w)*sl_sg + w*sl
# cnn_elev = (1-w)*cnn + w*sl

# # with h5py.File(REFCST_dir + "SL_final_SS_2019_lead{}.hdf".format(lead), 'r') as h5io:
# #     sl_elev_ = h5io['AnEn'][:, en, ...]

# # sl_elev = np.empty((365,)+land_mask_bc.shape)
# # sl_elev[...] = np.nan
# # sl_elev[..., ~land_mask_bc] = sl_elev_

# # with h5py.File(REFCST_dir + "SL_CNN_SS_2019_lead{}.hdf".format(lead), 'r') as h5io:
# #     cnn_elev_ = h5io['AnEn'][:, en, ...]

# # cnn_elev = np.empty((365,)+land_mask_bc.shape)
# # cnn_elev[...] = np.nan
# # cnn_elev[..., ~land_mask_bc] = cnn_elev_
    
# DATA = {}
# DATA['sl'] = 8*sl[day_plot, ...]
# DATA['era'] = 8*ERA_true[day_plot, ...]
# DATA['sl_sg'] = 8*sl_sg[day_plot, ...]
# DATA['sl_elev'] = 8*sl_elev[day_plot, ...]
# DATA['cnn'] = 8*cnn[day_plot, ...]
# DATA['cnn_elev'] = 8*cnn_elev[day_plot, ...]

# np.save(save_dir+'AnEn_example.npy', DATA)

In [6]:
DATA = np.load(save_dir+'AnEn_example.npy', allow_pickle=True)[()]

keys = ['sl', 'era', 'sl_sg', 'sl_elev', 'cnn', 'cnn_elev']

# Figure

In [7]:
# Cartopy map settings
scale_param = '50m' # 10m for publication quality
# US states and CAN-US boundary
PROVINCE = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale=scale_param,
    facecolor='none')

In [8]:
def aspc_cal(edge):
    return (edge[3]-edge[2])/(edge[1]-edge[0])

def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(b, a % b)

In [9]:
edge_bc = [-134-1.5, -114.5-0.5, 48.25, 60]
r_bc = aspc_cal(edge_bc)

In [10]:
cmap_pct, A = gu.precip_cmap()

/glade/work/ksha/py_env_20200417/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in greater_equal


In [11]:
VLIM = [0, 15*5]
hist_bins = np.arange(20, 65, 2)
gray = A[0, :]

fig = plt.figure(figsize=(13, 0.5*3*13*r_bc), dpi=dpi_)
gs = gridspec.GridSpec(3, 2, height_ratios=[1, 1, 1], width_ratios=[1, 1])
ax1 = plt.subplot(gs[0, 0], projection=ccrs.PlateCarree())
ax2 = plt.subplot(gs[0, 1], projection=ccrs.PlateCarree())
ax3 = plt.subplot(gs[1, 0], projection=ccrs.PlateCarree())
ax4 = plt.subplot(gs[1, 1], projection=ccrs.PlateCarree())
ax5 = plt.subplot(gs[2, 0], projection=ccrs.PlateCarree())
ax6 = plt.subplot(gs[2, 1], projection=ccrs.PlateCarree())

plt.subplots_adjust(0, 0, 1, 1, hspace=0, wspace=0)
AX = [ax1, ax2, ax3, ax4, ax5, ax6]

titles = [
    '(a) A direct output of AnEn with 19 supplemental locations (SLs)'+
    '\n     GEFS refcst init time: 0000 UTC 31 January 2019'+'\n     Forecast lead time: 12 hours', 
    '(b) The ERA5 reanalysis on the forecasted time     ', 
    '(c) Smoothed version of (a)\n     Produced by Savitzky–Golay (SG) filter', 
    '(d) Weighted combination of (a) and (c)                  \n',
    '(e) Denoised version of (a)\n     Produced by our CNN',
    '(f) Weighted combination of (a) and (e)                  \n']

#Weighted combination of (a) and (c) with quantile mapping
handle_text_title = []
yw = [0.9775]*6
xw = 0.0125
for n, ax in enumerate(AX):
    ax.set_extent(edge_bc, ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale(scale_param), edgecolor='k', linewidth=1.5)
    ax.add_feature(cfeature.BORDERS.with_scale(scale_param), linestyle='--', linewidth=2.5)
    ax.add_feature(PROVINCE, edgecolor='k', linestyle=':', linewidth=2.5)
    ax.spines['geo'].set_linewidth(2.5)
    handle_text_title.append(ax.text(xw, yw[n], titles[n], ha='left', va='top', 
                                     fontsize=14, transform=ax.transAxes, zorder=3))
handle_subtext = []    
handle_subtext += gu.string_partial_format(fig, ax4, xw, 0.915, 'left', 'top', 
                                           ['     (will be Schaake shuffled to produce ', 'SL-SG', ')'], 
                                           ['k', cyan, 'k'], [14,]*3,  ['normal', 'bold', 'normal'])
handle_subtext += gu.string_partial_format(fig, ax6, xw, 0.915, 'left', 'top', 
                                           ['     (will be Schaake shuffled to produce ', 'SL-CNN', ')'], 
                                           ['k', blue, 'k'], [14,]*3,  ['normal', 'bold', 'normal'])
LOCX = [0, 0.5]
LOCY = [2/3, 1/3, 0]
AX_hist = []
for locy in LOCY:
    for locx in LOCX:
        AX_hist.append(fig.add_axes([0.02+locx, 0.025+locy, 0.175, 0.085]))
        
for i, key in enumerate(keys): 
    CS = AX[i].pcolormesh(bc_lon, bc_lat, DATA[key], vmin=VLIM[0], vmax=VLIM[1], cmap=cmap_pct)
    AX_hist[i].spines["bottom"].set_visible(False)
    AX_hist[i].spines["top"].set_visible(False)
    AX_hist[i].spines["left"].set_visible(True)
    AX_hist[i].spines["right"].set_visible(False)
    [j.set_linewidth(2.5) for j in AX_hist[i].spines.values()]
    AX_hist[i].xaxis.set_tick_params(labelsize=12)
    AX_hist[i].yaxis.set_tick_params(labelsize=12)
    AX_hist[i].tick_params(axis="both", which="both", pad=0, bottom=False, top=False, 
                           labelbottom=True, left=False, right=False, labelleft=True)
    AX_hist[i].set_ylim([0, 101])
    AX_hist[i].ticklabel_format(axis='y', style='sci', scilimits=(0, 1))
    AX_hist[i].set_xlim([17.5, 65.01])
    AX_hist[i].set_xticks([20, 30, 40, 50, 65])
    AX_hist[i].set_xticklabels(['20', '30', '40', '50', '[mm/day]'])
    AX_hist[i].hist(DATA[key][DATA[key]>=15], bins=hist_bins, facecolor=gray, edgecolor='k', linewidth=1.5);
    AX_hist[i].patch.set_alpha(0)
    
for handle in handle_text_title:
    handle.set_bbox(dict(facecolor='w', edgecolor='none', zorder=4))

for handle in handle_subtext:
    handle.set_bbox(dict(facecolor='w', pad=0, edgecolor='none', zorder=5))
    
ax_base = fig.add_axes([0.935, 5/6-0.03, 0.02, 1/6+0.02])
[j.set_linewidth(0) for j in ax_base.spines.values()]
ax_base.tick_params(axis='both', left=False, top=False, right=False, bottom=False, \
                    labelleft=False, labeltop=False, labelright=False, labelbottom=False)
cax = inset_axes(ax_base, height='100%', width='50%', borderpad=0, loc=2)
CBar = plt.colorbar(CS, orientation='vertical', extend='max', cax=cax)
CBar.ax.tick_params(axis='y', labelsize=14, direction='in', length=0)
CBar.set_label('precip rate [mm/day]', fontsize=14)
CBar.outline.set_linewidth(2.5)

ax_w1 = fig.add_axes([0.025, 2/3+0.075, 0.1, 0.03])
ax_w1.set_axis_off()
ax_w1.text(0, 1, '1D histogram\nwith bin counts', fontsize=12, ha='left', va='top', transform=ax_w1.transAxes);

ax_w2 = fig.add_axes([0.9, 0.835, 0.03, 0.125])
ax_w2.set_axis_off()
ax_w2.text(0.5, 1, '5 [mm/day]\nper color scale', fontsize=14, 
           rotation=90, ha='center', va='top', transform=ax_w2.transAxes);

if need_publish:
    # Save figure
    fig.savefig(fig_dir+'AnEn_example.png', format='png', **fig_keys)